<a href="https://colab.research.google.com/github/Niraj-Khatri/Pyspark-AWS/blob/master/VideoGames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Spark and Import Dataset

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-07 14:16:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.10MB/s    in 0.2s    

2021-09-07 14:16:31 (5.10 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Import Amazon Electronics Data File From AWS


In [4]:
# Import struct fields
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, TimestampType

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
data = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep='\t', header=True)

# Clean the data
data = data.withColumn('review_date', data['review_date'].cast(TimestampType()))
data = data.withColumn('customer_id', data['customer_id'].cast(IntegerType()))
data = data.withColumn('product_parent', data['product_parent'].cast(IntegerType()))
data = data.withColumn('star_rating', data['star_rating'].cast(IntegerType()))
data = data.withColumn('helpful_votes', data['helpful_votes'].cast(IntegerType()))
data = data.withColumn('total_votes', data['total_votes'].cast(IntegerType()))
df = data.dropna()

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

# Create Tables

## Customers Table

In [6]:
# Customer ID with customer view count
customers = df.groupBy("customer_id").count()
customers = customers.withColumnRenamed('count','customer_count')
customers = customers.withColumn('customer_count', customers['customer_count'].cast(IntegerType()))


# Products Table

In [7]:
# Product ID and product title
products = df.groupby("product_id", "product_title").count().select("product_id", "product_title")


## Reviews Table

In [8]:
# The review number, who reviewed it, what he/she reviewed, the parent of the reviewed product, and the date of the review
reviews = df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")

## Vines Table

In [9]:
# The review number, rating, helpful votes, total votes, and vine 
vines = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")

# Connect To RDS

In [10]:
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://amazonreviews.c5bki1uyc90h.us-east-2.rds.amazonaws.com:5432/AmazonReviews"
config = {"user":"postgres", 
          "password": "Goldensun123!", 
          "driver":"org.postgresql.Driver"}

In [11]:
# Write tables to postgres in RDS
customers.write.jdbc(url=jdbc_url, table = "customers", mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table = "products", mode=mode, properties=config)
reviews.write.jdbc(url=jdbc_url, table = "review_id_table", mode=mode, properties=config)
vines.write.jdbc(url=jdbc_url, table = "vine_table", mode=mode, properties=config)

# Data Analysis
#### In this section, I will analyze the Amazon video game data to determine if vine reviewers provide more helpful reviews

In [28]:
# Filter reviews to only include reviews that had greater than 50% of helpful votes
# Filter reviews with more than 20 total votes
vines.filter((vines.helpful_votes/vines.total_votes >= .5) & (vines.total_votes >= 20)).show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R4PKAZRQJJX14|          1|           21|         34|   N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|
|R127WEQY2FM1T3|          1|          147|        175|   N|
|R2FJ94555FZH32|          2|           55|         60|   N|
|R1U3AR67RE273L|          1|           51|         65|   N|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|
|R36O341WWXXKNP|          5|           28|         31|   N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|
|R2OWDAR0EDEPTF|          1|           23|         28|   N|
|R29BOS5HMAY1LO|          5|           88|        110|   N|
| R8WD0136CETA7|          2|           99|        119|   N|
|R16621F16PQN5A|          3|           2

In [25]:
# 